In [ ]:
import os

import pandas as pd

from adversarial import RandomAdversarialAlignments
from utils import load_config


config = load_config()

languages = ['en']

In [ ]:
if not os.path.exists('data/atis/adversarial'):
    os.mkdir('data/atis/adversarial')

for base_language in languages:
    other_languages = list(config['languages'])
    other_languages.remove(base_language)

    dataset = pd.DataFrame()

    for language in other_languages:
        dataset = pd.concat(
            (
                dataset,
                RandomAdversarialAlignments(
                    base_language=base_language,
                    languages=[language],
                    subset='train',
                    num_examples=1
                ).generate_dataset('train')
            )
        )

    dataset.to_csv(f'data/atis/adversarial/adversarial_{base_language}.csv')

In [ ]:
import numpy as np


uuids = set()

for base_language in languages:
    dataset = pd.read_csv(f'data/atis/adversarial/adversarial_{base_language}.csv')

    dataset['uuid'] = np.arange(len(dataset))
    dataset['len'] = dataset['utterance'].str.split().apply(lambda x: len(x))

    uuids.update(dataset.loc[dataset['len'] >= 30]['uuid'])

In [ ]:
for base_language in languages:
    dataset = pd.read_csv(f'data/atis/adversarial/adversarial_{base_language}.csv')
    dataset['uuid'] = np.arange(len(dataset))

    dataset = dataset.loc[~dataset['uuid'].isin(uuids)]
    dataset.drop('uuid', axis=1, inplace=True)

    dataset.to_csv(f'data/atis/adversarial/adversarial_{base_language}.csv')